<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark_streaming/dataproc/producer_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Producer simulation

- To be executed in Google Colab
- Connect to GCLOUD
- Fetch data from API (Carris, vehicles endpoint)
- Copy data to GC bucket

In [2]:
from google.colab import auth
auth.authenticate_user()

project_id = 'data-eng-dev-437916'
!gcloud config set project {project_id}

Updated property [core/project].


In [11]:
!mkdir -p /content/temp/

In [ ]:
import requests
from pyspark.sql.types import *
import json
import datetime
import time

def ingest_from_api(url: str, landing_path: str):
    print(f"fetching data from {url}...")
    response = requests.get(url)
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    temp_file = f"/content/temp/vehicles_{int(timestamp)}.json"

    if response.status_code == 200:
        data = response.json()
        with open(temp_file, "w") as f:
            json.dump(data, f)
            print("Writing into temp location...")
            !gsutil cp {temp_file} {landing_path}

def producer_vehicles(loop: int, interval_time: int, landing_path: str):
    for i in range(loop):
        print(f"Producer running...{i}")
        ingest_from_api(f"https://api.carrismetropolitana.pt/vehicles", landing_path)
        time.sleep(interval_time)

if __name__ == '__main__':
  print("# Starting process #")
  bucket_name="edit-data-eng-dev"
  table="vehicles"
  landing_path=f"gs://{bucket_name}/datalake/landing/{table}"
  print(f"Landing path: {landing_path}")
  producer_vehicles(10, 30, landing_path)
  print("# Process done #")

# Starting process #
Landing path: gs://edit-data-eng-dev/datalake/landing/vehicles
Producer running...0
fetching data from https://api.carrismetropolitana.pt/vehicles...
Writing into temp location...
Copying file:///content/temp/vehicles_20241129141154.json [Content-Type=application/json]...
-
Operation completed over 1 objects/287.8 KiB.                                    
Producer running...1
fetching data from https://api.carrismetropolitana.pt/vehicles...
Writing into temp location...
Copying file:///content/temp/vehicles_20241129141229.json [Content-Type=application/json]...
-
Operation completed over 1 objects/287.8 KiB.                                    
Producer running...2
fetching data from https://api.carrismetropolitana.pt/vehicles...
Writing into temp location...
Copying file:///content/temp/vehicles_20241129141304.json [Content-Type=application/json]...
-
Operation completed over 1 objects/279.9 KiB.                                    
Producer running...3
fetching data